In [ ]:
base_dir = '/glade/work/kjmayer/research/catalyst/S2S_ocn_lnd_atm/'

In [1]:
import xarray
import numpy
import torch
from torch import nn
from torch.utils.data import DataLoader

In [3]:
import sys
sys.path.append(base_dir+'functions/')
from train_utils import NeuralNetwork, train, val

In [ ]:
def set_gpu(gpu_id):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    
gpu_id=0

if gpu_id >= 0:
    device = "cuda"
    set_gpu(gpu_id)
    print('device available :', torch.cuda.is_available())
    print('device count: ', torch.cuda.device_count())
    print('current device: ',torch.cuda.current_device())
    print('device name: ',torch.cuda.get_device_name())
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('cpu')

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

print('using device: ', device)

In [ ]:
class MakeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.nan_to_num(torch.tensor(X[randi], dtype=torch.float32))
        self.y = torch.tensor(np.mean(y[randi,ilat:flat,ilon:flon],axis=(1,2)), dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].unsqueeze(0), self.y[idx].unsqueeze(0)

In [ ]:
EPOCHS = 5
BATCH_SIZE = 64
LR = 1e-3

model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

In [ ]:
## Training Data!
training_data = SpatialDataset(np_arr_x_train_anom, np_arr_y_train_anom )
train_dataloader = DataLoader(training_data,batch_size = BATCH_SIZE)

## Validation Data!
val_data = SpatialDataset(np_arr_x_val_anom, np_arr_y_val_anom)
val_dataloader  = DataLoader(val_data,batch_size = BATCH_SIZE)



In [ ]:
for e in range(EPOCHS):
    print(f"Epoch {e+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("Done!")